In [65]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [66]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, unix_timestamp, lit
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.window import Window

In [67]:
def ipv4_to_32bit_int(ipv4):
    a, b, c, d = [int(x) for x in ipv4.split('.')]
    return (a << 24) + (b << 16) + (c << 8) + d


def interleave_bits(x, y):
    z = 0
    for i in range(16):
        z |= (x & 1 << i) << i | (y & 1 << i) << (i + 1)
    return z


def z_curve_mapping(ipv4):
    ip_int = ipv4_to_32bit_int(ipv4)
    x = ip_int & 0xAAAAAAAA
    y = ip_int & 0x55555555
    x = (x | (x >> 1)) & 0x33333333
    x = (x | (x >> 2)) & 0x0F0F0F0F
    x = (x | (x >> 4)) & 0x00FF00FF
    x = (x | (x >> 8)) & 0x0000FFFF
    y = (y | (y >> 1)) & 0x33333333
    y = (y | (y >> 2)) & 0x0F0F0F0F
    y = (y | (y >> 4)) & 0x00FF00FF
    y = (y | (y >> 8)) & 0x0000FFFF
    morton_number = interleave_bits(x, y)
    lat = (morton_number / (2 ** 32 - 1)) * 180 - 90
    lng = (morton_number % (2 ** 32 - 1)) * 360 / (2 ** 32 - 1) - 180
    return round(lat, 5), round(lng, 5)

In [68]:
spark = SparkSession.builder \
    .appName("Add Lat and Lon to DataFrame") \
    .getOrCreate()

schema = StructType([
    StructField("src_ip", StringType(), True),
    StructField("dest_ip", StringType(), True),
    StructField("timestamp", StringType(), True)
])

In [69]:
input_file = "requests.csv"
df = spark.read.csv(input_file, header=True, schema=schema)

def safe_z_curve_mapping(src_ip):
    try:
        lat, lon = z_curve_mapping(src_ip)
        return lat, lon
    except Exception as e:
        return None, None
    
safe_z_curve_mapping_udf = udf(safe_z_curve_mapping, returnType=StructType([
    StructField("lat", DoubleType(), True),
    StructField("lon", DoubleType(), True)
]))

In [70]:
df_with_lat_lon = df.withColumn("lat_lon", safe_z_curve_mapping_udf(col("src_ip")))
df_with_lat_lon = df_with_lat_lon.select("src_ip", "dest_ip", "timestamp", "lat_lon.lat", "lat_lon.lon")

In [71]:
df_filtered = df_with_lat_lon.dropna()
df = df_filtered.drop("src_ip")

In [72]:

# Step 1: Sort the DataFrame by the 'timestamp' column
df = df.orderBy('timestamp')

# Step 2: Convert the 'timestamp' column to timestamp format
df = df.withColumn('timestamp', col('timestamp').cast('timestamp'))

# Step 3: Calculate time differences between each row and the first row
window = Window.orderBy('timestamp')
first_timestamp = df.select('timestamp').first()[0]
first_timestamp_lit = lit(first_timestamp)
df = df.withColumn('timestamp_delta', (unix_timestamp(col('timestamp')) - unix_timestamp(lit(first_timestamp))) * 10)

# Print the updated DataFrame
df.show()

23/05/08 01:44:29 WARN TransportChannelHandler: Exception in connection from /10.251.192.224:49236
java.io.IOException: Operation timed out
	at java.base/sun.nio.ch.SocketDispatcher.read0(Native Method)
	at java.base/sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:47)
	at java.base/sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:330)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:284)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:259)
	at java.base/sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:417)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:259)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:357)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:788)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOpti

Py4JJavaError: An error occurred while calling o234.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 38.0 failed 1 times, most recent failure: Lost task 0.0 in stage 38.0 (TID 183) (10.251.192.224 executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2785)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2721)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2720)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2720)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DA

In [63]:
df = df.drop("timestamp")

In [64]:
output_dir = "output"
df = df.repartition(1)
df.write.csv(output_dir, header=True, mode="overwrite")